<a href="https://colab.research.google.com/github/sthalles/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/Part%204%20-%20Fashion-MNIST%20(Exercises).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Fashion-MNIST

Now it's your turn to build and train a neural network. You'll be using the [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist), a drop-in replacement for the MNIST dataset. MNIST is actually quite trivial with neural networks where you can easily achieve better than 97% accuracy. Fashion-MNIST is a set of 28x28 greyscale images of clothes. It's more complex than MNIST, so it's a better representation of the actual performance of your network, and a better representation of datasets you'll use in the real world.

<img src='https://github.com/sthalles/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/fashion-mnist-sprite.png?raw=1' width=500px>

In this notebook, you'll build your own neural network. For the most part, you could just copy and paste the code from Part 3, but you wouldn't be learning. It's important for you to write the code yourself and get it to work. Feel free to consult the previous notebooks though as you work through this.

First off, let's load the dataset through torchvision.

In [0]:
import torch
from torchvision import datasets, transforms
import helper

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Here we can see one of the images.

In [2]:
image, label = next(iter(trainloader))
print(image.shape)
print(label.shape)
# helper.imshow(image[0,:]);

torch.Size([64, 1, 28, 28])
torch.Size([64])


## Building the network

Here you should define your network. As with MNIST, each image is 28x28 which is a total of 784 pixels, and there are 10 classes. You should include at least one hidden layer. We suggest you use ReLU activations for the layers and to return the logits or log-softmax from the forward pass. It's up to you how many layers you add and the size of those layers.

In [0]:
from torch import nn
import torch.nn.functional as F

# TODO: Define your network architecture here
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784,256)
    self.fc2 = nn.Linear(256,64)
    self.fc3 = nn.Linear(64,10)

  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)
    return x

# Train the network

Now you should create your network and train it. First you'll want to define [the criterion](http://pytorch.org/docs/master/nn.html#loss-functions) ( something like `nn.CrossEntropyLoss`) and [the optimizer](http://pytorch.org/docs/master/optim.html) (typically `optim.SGD` or `optim.Adam`).

Then write the training code. Remember the training pass is a fairly straightforward process:

* Make a forward pass through the network to get the logits 
* Use the logits to calculate the loss
* Perform a backward pass through the network with `loss.backward()` to calculate the gradients
* Take a step with the optimizer to update the weights

By adjusting the hyperparameters (hidden units, learning rate, etc), you should be able to get the training loss below 0.4.

In [0]:
# TODO: Create the network, define the criterion and optimizer
from torch import optim

model = Network()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [5]:
# TODO: Train the network here
epochs = 5

for i in range(epochs):
  running_loss = 0
  for x_batch, y_batch in trainloader:

    x_batch = x_batch.view(x_batch.shape[0], -1)
    logprobs = model(x_batch)

    # compute the loss
    loss = criterion(logprobs, y_batch)

    # zero the gradeints before computing the gradients
    optimizer.zero_grad()     

    # compute the gradients
    loss.backward()

    # perform grandient descent step
    optimizer.step()
    running_loss += loss.item()

  print("End of epoch {}, Loss: {}, Running loss: {}".format(i, loss, running_loss))

End of epoch 0, Loss: 0.3415629267692566, Running loss: 536.1184098422527
End of epoch 1, Loss: 0.25163504481315613, Running loss: 375.93359953165054
End of epoch 2, Loss: 0.16836291551589966, Running loss: 336.7297277674079
End of epoch 3, Loss: 0.33789435029029846, Running loss: 311.79332357645035
End of epoch 4, Loss: 0.1254456341266632, Running loss: 292.1881694123149


In [6]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import helper

# Test out your network!

dataiter = iter(testloader)
images, labels = dataiter.next()
print(images.shape)

# TODO: Calculate the class probabilities (softmax) for img
with torch.no_grad():
  images = images.view(images.shape[0], -1)
  logprobs = model(images)

ps = torch.exp(logprobs)
print("probs shape:", ps.shape)

# Plot the image and probabilities
# helper.view_classify(img.resize_(1, 28, 28), ps, version='Fashion')
preds = torch.argmax(ps, dim=1)

torch.Size([64, 1, 28, 28])
probs shape: torch.Size([64, 10])


In [9]:
print("Accuracy {}".format(torch.mean((preds == labels).double())))

Accuracy 0.828125
